<a href="https://colab.research.google.com/github/MahmoodAbdali79/Face-ani-spoofing/blob/main/benchmark/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://keras.io/examples/vision/video_classification/
# https://keras.io/api/preprocessing/image/
# https://discuss.pytorch.org/t/how-to-go-about-unbalanced-data-in-a-binary-classification-in-pytorch/90655

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/shenasa/benchmark

Mounted at /content/drive
/content/drive/MyDrive/shenasa/benchmark


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.preprocessing.image import img_to_array
from rPPG.rPPG_Extracter import *
from rPPG.rPPG_lukas_Extracter import *
from keras.models import model_from_json
import tensorflow as tf
from random import seed
# from random import randint
from random import sample

seed(1)

In [ ]:
dim = (128,128)

In [ ]:
cascPath = './cv2_model/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

In [ ]:
# extract frames from videos (each 5 frames)
# numberfolder_filemname_x,y,w,h_label.jpg

def extract_frames(path, max_frames=6):
  folders = os.listdir(path)
  os.mkdir(f"{path}/frames")
  
  for folder in folders:
    print(f'exracting foder {folder} ...')
    videos = os.listdir(f'{path}/{folder}')
    for video in videos:
      print(video.split('.')[0], ' ...')
      name = video.split('.')[0]
      if name in ['1','2','HR_1'] : label = '1'
      else : label = '0'
      cap = cv2.VideoCapture(f'{path}/{folder}/{video}')
      
      try:
        i = 0 
        counter = 0
        while True:
          ret, frame = cap.read()
          if not ret: break

          i += 1
          # detect face 
          gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)


          if i%5 == 0 and len(faces) != 0:
            x, y, w, h = faces[0]
            print( faces[0],len(faces[0]) , f'counter: {counter+1}')
            cv2.imwrite(f'{path}/frames/{folder}_{name}_{x},{y},{w},{h}_{label}.jpg', frame)
            counter += 1
          # if counter == max_frames: break
      finally:
        cap.release()

In [ ]:
extract_frames('test_release')

In [ ]:
def get_rppg_pred(frame):
    use_classifier = True  
                                 
    sub_roi = []           
    use_resampling = False  
    
    fs = 20

    timestamps = []
    time_start = [0]

    break_ = False

    rPPG_extracter = rPPG_Extracter()
    rPPG_Lukas_Extracter()
    bpm = 0
    
    dt = time.time()-time_start[0]
    time_start[0] = time.time()
    if len(timestamps) == 0:
        timestamps.append(0)
    else:
        timestamps.append(timestamps[-1] + dt)
        

    rPPG_extracter.measure_rPPG(frame,use_classifier,sub_roi) 
    rPPG = np.transpose(rPPG_extracter.rPPG)
    
        # Extract Pulse
    if rPPG.shape[1] > 10:
        if use_resampling :
            t = np.arange(0,timestamps[-1],1/fs)
            
            rPPG_resampled= np.zeros((3,t.shape[0]))
            for col in [0,1,2]:
                rPPG_resampled[col] = np.interp(t,timestamps,rPPG[col])
            rPPG = rPPG_resampled
        num_frames = rPPG.shape[1]

        
    return rPPG

In [ ]:
def make_pred(li):
    [single_img,rppg] = li
    # print(single_img.shape)
    single_img = cv2.resize(single_img, dim)
    single_x = img_to_array(single_img)
    single_x = np.expand_dims(single_x, axis=0)

    # print(single_img.shape)
    # print('============')
    # print(single_x.shape)
    # print('============')
    # print(rppg.shape)

    single_pred = model.predict([single_x,rppg])
    return single_pred

In [ ]:
def prediction_test(path):
  result = []
  label = []
  ma_frames = os.listdir(f'{path}/frames')
  counter = 0
  for i in ma_frames:
    print(f'{counter}:', i)
    im = cv2.imread(f'{path}/frames/{i}')
    if 'HR' in i:
      face = list(map(int, i.split('_')[3].split(',')))
    else:
      face = list(map(int, i.split('_')[2].split(',')))

    x, y, w, h = face
    sub_img=im[y:y+h,x:x+w]
    
    rppg_s = get_rppg_pred(sub_img)
    rppg_s = rppg_s.T
    # print(sub_img.shape)
    # print(rppg_s)
    pred = make_pred([sub_img,rppg_s])
    result.append(pred)
    label.append(int(i.split('_')[-1].split('.')[0]))
    counter += 1

  result = np.array(result).reshape(len(result), -1) 
  result = tf.math.argmin(result, axis=1)
  label = tf.convert_to_tensor(label)

  return result, label
  


def PredictSepratedClass(class_folder):
  result = []
  label = []

  frames = os.listdir('benchmark_type/'+class_folder)
  # print(frames[:10])

  for i, frame in enumerate(frames):
    print(f'Counter {i} ... ')
    img = cv2.imread(f'benchmark_type/{class_folder}/{frame}')
    if 'HR' in frame:
      x, y, w, h = list(map(int,frame.split('_')[3].split(',')))
    else:
      x, y, w, h = list(map(int,frame.split('_')[2].split(',')))
    sub_img = img[y:y+h,x:x+w]

    rppg_s = get_rppg_pred(sub_img)
    rppg_s = rppg_s.T
    pred = make_pred([sub_img,rppg_s])
    result.append(pred)
    label.append(int(frame.split('_')[-2]))

  result = np.array(result).reshape(len(result), -1) 
  result = tf.math.argmin(result, axis=1)
  label = tf.convert_to_tensor(label)

  return result, label




In [ ]:
json_file = open('../RGB_rPPG_merge_softmax_.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("../balance_30_300_7.h5")

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
result, label = prediction_test('test_release')

In [ ]:
len(os.listdir('test_release/frames'))

8950

In [ ]:
label.shape

TensorShape([8950])

In [ ]:
res = tf.math.confusion_matrix(label,result)
print(res)

tf.Tensor(
[[4829 1364]
 [1180 1577]], shape=(2, 2), dtype=int32)


In [ ]:
# test_30_180_1  first method : 0.29465575349061146
# (1083+141)/(2*2077)

# test_30_180_1 second method : 0.29809955540057537
# (6087+752)/(2*11471)

# test_30_180_1 third method : 0.29
# (4439+752)/(2*8950)

0.29

In [ ]:
# balance_30_300_7 first method : 0.36398651901781415
# (1504+8)/(2*2077)

# balance_30_300_7 second method : 0.3687124051957109
# (8346+113)/(2*11471)

# balance_30_300_7 third method : 0.26261877778746406
# (5912+113)/(2*11471)

0.26261877778746406

In [ ]:
# RGB_rPPG_merge_softmax_   first method : 0.343042850264805
# (1129+296)/(2*2077) 

# RGB_rPPG_merge_softmax_   second method : 0.3496643710225787
# (6445+1577)/(2*11471)

# RGB_rPPG_merge_softmax_   third method : 0.27922587394298665
# (4829+1577)/(2*11471)

In [ ]:
(2077)/(2*2077)

0.5

In [ ]:
(0)/(2*2077)

0.0

In [ ]:
files = os.listdir('test_release/frames')

In [ ]:
a = files[100]
print(a)

if 'HR' in a: print('_'.join(a.split('_')[1:3]))
else: print('_'.join(a.split('_')[1]))

15_HR_2_37,292,635,635_0.jpg
HR_2


In [ ]:
for f in files:
  if 'HR' in f: name = '_'.join(f.split('_')[1:3])
  else: name = '_'.join(f.split('_')[1])

  if name in ['5', '6', 'HR_3']:
    try:
      # print(f)
      os.remove('test_release/frames/'+f)
    except:pass

In [ ]:
len( os.listdir('test_release/frames'))

8950

In [ ]:
try:
  os.remove('test_release/frames/'+files[10])
except:pass

In [ ]:
def extract_frame(in_paht):  #test_release/18/3.avi
  #extrame frae from a path and release in a other path
  folder = in_paht.split('/')[1] 
  name = in_paht.split('.')[0].split('/')[-1]
  label = '0'
  number_frame = 10
  out_folder = '' 

  if name in ['1','2','HR_1'] : 
    label = '1'
    number_frame = 30
    out_folder = 'real'
  elif name in ['3', '4', 'HR_2']:
    out_folder = 'print'
  elif name in ['7', '8', 'HR_4']:
    out_folder = 'reply'
  elif name in ['5', '6', 'HR_3']:
    out_folder = 'print_eye'

  print(f'    Extracting {name}.avi ...')

  cap = cv2.VideoCapture(in_paht)
  length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  selected_frame = np.sort(sample(range(length), number_frame))
  last_frame = selected_frame[-1]

  A = []

  # print(selected_frame, len(selected_frame))

  try:
    i = 0 
    while True:
      ret, frame = cap.read()
      if not ret: break

      # detect face
      try:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        if i in selected_frame and len(faces) != 0:
          A.append(i)
          # print(f'in frame : {i}')
          x, y, w, h = faces[0]
          cv2.imwrite(f'benchmark_type/{out_folder}/{folder}_{name}_{x},{y},{w},{h}_{label}_{i}.jpg', frame)
      
        i += 1
      except Exception as e:
        print(f'can not write frame {i}')
        print(e)
      if i == last_frame+1:
        break

  finally:
    cap.release()


In [ ]:
def ExtractFrameBasedOnClass(path):
  folders = os.listdir(path)
  
  for folder in folders:
    print(f'On folder {folder} ...')
    videos = os.listdir(f'{path}/{folder}')
    for video in videos:
      extract_frame(f'{path}/{folder}/{video}')

In [ ]:
# !rm benchmark_type/reply/*
# !ls benchmark_type/real

In [ ]:
ExtractFrameBasedOnClass('test_release')

In [ ]:
# real - atack
2576-2488

88

In [ ]:
print(len(os.listdir('benchmark_type/print')))
print(len(os.listdir('benchmark_type/reply')))
print(len(os.listdir('benchmark_type/print_eye')))

818
826
844


In [ ]:
print(len(os.listdir('benchmark_type/real')))

2488


In [ ]:
print(len(os.listdir('benchmark_type/print')))

818


In [ ]:
pics = os.listdir('benchmark_type/real')

for i in sample(range(2576+1), 88):
  os.remove(f'benchmark_type/real/{pics[i]}')

In [ ]:
result, label = PredictSepratedClass('reply')

In [ ]:
res = tf.math.confusion_matrix(label,result)
print(res)

tf.Tensor(
[[769  57]
 [  0   0]], shape=(2, 2), dtype=int32)


In [ ]:
#@title an evaluation with a balance aproch


In [ ]:
# balance_30_300_7 for just real 0.011655948553054662
# (0+58)/(2*2488)

# test_30_180_1 for just reply 0.4654963680387409
# (769+0)/(2*826)

# test_30_180_1 for just print_eye 0.4786729857819905
# (808+0)/(2*844)

# test_30_180_1 for just print 0.4883863080684597
# (799+0)/(2*818)

In [ ]:
# RGB_rPPG_merge_softmax_ for just real 0.28516881028938906
# (0+1419)/(2*2488)

# RGB_rPPG_merge_softmax_ for just reply 0.3898305084745763
# (644+0)/(2*826)

# RGB_rPPG_merge_softmax_ for just print_eye 0.2950236966824645
# (498+0)/(2*844)

# RGB_rPPG_merge_softmax_ for just print 0.39425427872860636
# (645+0)/(2*818)

0.39425427872860636

In [ ]:
# test_30_180_1 for just real 0.12781350482315113
# (0+636)/(2*2488)

# test_30_180_1 for just reply 0.2950236966824645
# (618+0)/(2*826)

# test_30_180_1 for just print_eye 0.33293838862559244
# (562+0)/(2*844)

# test_30_180_1 for just print 0.3471882640586797
# (568+0)/(2*818)

0.12781350482315113

In [ ]:
# RGB_rPPG_merge_softmax_ for just reply 0.3898305084745763
(498+0)/(2*844)

0.2950236966824645

In [ ]:
os.listdir('benchmark_type/real')[40:50]

In [ ]:
'26_HR_1_8,450,648,648_1_65.jpg'.split('_')[3].split(',')

['8', '450', '648', '648']

In [ ]:
#@title new balance for evaluation


In [ ]:
def balanced_film_name():
  real = ['1', '2', 'HR_1'] 
  print = ['3', '4', 'HR_2']
  print_eye = ['5', '6', 'HR_3'] 
  reply = ['7', '8', 'HR_4' ] 

  print = np.random.choice(print, 1)
  print_eye = np.random.choice(print_eye, 1)
  reply = np.random.choice(reply, 1)

  D = np.concatenate([print,print_eye,reply,real])

  return D

In [ ]:
for file_name in list(balanced_film_name()):
  if file_name in ['1', '2', 'HR_1']:
    print(file_name)

1
2
HR_1


In [ ]:
# real : 3
# atack : 9

# ==> 3 real   <==>   1 reply  1 print  1 print_eye


# 6×15×30   number film(per folder) * number frame * nnumber folder


def extract_balanced_frames(path, number_frame):
  folders = os.listdir(path)
  for folder in folders:
    print(folder)
    for file_name in list(balanced_film_name()):
      video = f'{path}/{folder}/{file_name}.avi'
      cap = cv2.VideoCapture(video)
      length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
      selected_frame = np.sort(sample(range(length), number_frame))
      last_frame = selected_frame[-1]


      label = '0'
      if file_name in ['1', '2', 'HR_1']:
        label = '1'

      i = 0

      while True:
        i += 1
        ret, frame = cap.read()
        if not ret: break

        try:
          gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

          if i in selected_frame and len(faces) != 0:
            # print(f'in frame : {i}')
            x, y, w, h = faces[0]
            cv2.imwrite(f'frame_balenced/{folder}_{file_name}_{x},{y},{w},{h}_{label}.jpg', frame)    # add countre 
            print(f'    {folder}_{file_name}_{x},{y},{w},{h}_{label}.jpg')

        except Exception as e:
          print(f'can not write frame {i}')
          print(e)

      


In [ ]:
['1', '2', 'HR_1'] 
['3', '4', 'HR_2']
['5', '6', 'HR_3'] 
['7', '8', 'HR_4' ] 


In [ ]:
extract_balanced_frames("test_release", 15)

18
    18_4_112,146,229,229_0.jpg
    18_4_126,145,209,209_0.jpg
    18_4_130,147,206,206_0.jpg
    18_4_128,146,209,209_0.jpg
    18_4_121,137,227,227_0.jpg
    18_4_131,144,215,215_0.jpg
    18_4_128,142,225,225_0.jpg
    18_4_126,141,231,231_0.jpg
    18_4_127,142,230,230_0.jpg
    18_4_121,152,238,238_0.jpg
    18_4_125,157,236,236_0.jpg
    18_4_126,153,231,231_0.jpg
    18_4_124,133,231,231_0.jpg
    18_4_123,151,235,235_0.jpg
    18_4_129,133,231,231_0.jpg
    18_5_273,170,218,218_0.jpg
    18_5_273,173,216,216_0.jpg
    18_5_264,170,230,230_0.jpg
    18_5_267,172,224,224_0.jpg
    18_5_271,173,223,223_0.jpg
    18_5_262,170,234,234_0.jpg
    18_5_262,164,238,238_0.jpg
    18_5_265,175,231,231_0.jpg
    18_5_269,176,226,226_0.jpg
    18_5_268,179,227,227_0.jpg
    18_5_268,167,229,229_0.jpg
    18_5_266,165,234,234_0.jpg
    18_5_267,175,220,220_0.jpg
    18_7_192,161,217,217_0.jpg
    18_7_175,48,252,252_0.jpg
    18_7_187,51,243,243_0.jpg
    18_7_187,54,238,238_0.jpg
    18_7

In [ ]:
len(os.listdir('frame_balenced'))

2392

In [ ]:
!ls without_eye_print

HR  NOHR


In [ ]:
!rm HR_NOHR/NOHR/*

In [ ]:
# Separation of high rezolution from low rezolution
# adding more frame for real film

def seprate_HR():
  folders = os.listdir('test_release')
  for folder in ['23']:# for folder in folders:
    files = os.listdir(f'test_release/{folder}')
    for file_name in ['8.avi']:# for file_name in files:
      # if file_name in ['6.avi', '5.avi', 'HR_3.avi']: continue
      # print(f)

      sub_folder = 'NOHR'
      if file_name in ['HR_1.avi', 'HR_2.avi', 'HR_4.avi', 'HR_3.avi']:
        sub_folder = 'HR'

      label = '0'
      number_frame = 15
      if file_name in ['1.avi', '2.avi', 'HR_1.avi']:
        label = '1'
        number_frame = 30

      cap = cv2.VideoCapture(f'test_release/{folder}/{file_name}')
      
      frames = []
      faces = []

      i = 1 

      while True:
        ret, frame = cap.read()
        if not ret: break

        i+=1
        try:
          gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          face = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
          if len(face) != 0:
            faces.append(face[0])
            frames.append(frame)
        except Exception as e:
          print(f'can not write frame {i}')
          print(e)
      
      faces = np.array(faces)
      frames = np.array(frames)

      length = faces.shape[0]
      print(frames.shape, faces.shape, length)
      # selected_frame = np.sort(sample(range(1, length), number_frame))

      # selected_faces = faces[selected_frame]
      # selected_frames = frames[selected_frame]

      # for i, f in enumerate(selected_frames):
      #   x, y, w, h = faces[i]
      #   cv2.imwrite(f'HR_NOHR/{sub_folder}/{folder}_{file_name.split(".")[0]}_{x},{y},{w},{h}_n{i}_{label}.jpg', f)    # add countre 
      #   print(f'  {folder}_{file_name.split(".")[0]}_{x},{y},{w},{h}_n{i}_{label}.jpg')
      
      

In [ ]:
seprate_HR()

(186, 640, 480, 3) (186, 4) 186


In [ ]:
os.listdir('test_release')

['18',
 '11',
 '27',
 '25',
 '21',
 '23',
 '6',
 '5',
 '19',
 '7',
 '10',
 '22',
 '4',
 '29',
 '9',
 '17',
 '2',
 '1',
 '26',
 '16',
 '30',
 '12',
 '8',
 '3',
 '20',
 '24',
 '28',
 '15',
 '13',
 '14']

In [ ]:
len(os.listdir('without_eye_print/HR'))

90